<a href="https://colab.research.google.com/github/owenmk/ok_cars/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1
edited again